In [ ]:
!pip install -U gdown 
!pip install torchmetrics
!gdown --id 1n-YeSOxjOOKyhaU1OE3Bqp_HmDm3pIqq

In [ ]:
import torch 
import torch.nn as nn
import pandas as pd    
import numpy as np

In [ ]:
df = pd.read_csv("/content/file.csv")

In [ ]:
list(df)

In [ ]:
df_data = df[["TimeProcess","People", "H", "Positive", "Negative", "NH3"]]
df_label = df["label"].values.astype(np.long)

In [ ]:
df_data.head()

In [ ]:
df_data.values 

In [ ]:
# df_data = df_data.

In [ ]:
df_data.head()

In [ ]:
df_label

In [ ]:
df_data = (df_data-df_data.std(0)) / df_data.mean(0)

In [ ]:
df_data.to_csv("data.csv", index=False)

In [ ]:
!cat data.csv | head

In [ ]:
len(df_data)

In [ ]:
df_data_train, df_label_train = df_data[:-1848], df_label[:-1848]
df_data_valid, df_label_valid = df_data[-1848:], df_label[-1848:]

In [ ]:
# len(df_data_train), len(df_label_train)

In [ ]:
df_data_valid

In [ ]:
from torch.utils.data import Dataset
import numpy as np
class DataVS(Dataset):
    def __init__(self, df, label):
        self.df = df.values.astype(np.float32)
        self.df = self.df.reshape((-1,12,6))
        self.label = label.reshape((-1,12))
        # print(self.df.shape)
        # print(self.label.shape)
    def __getitem__(self, idx):
        x = torch.from_numpy(self.df[idx][:,2:4])
        y = self.label[idx][-1]
        return x, y
    def __len__(self):
        return len(self.df)

data_vs_train = DataVS(df_data_train, df_label_train)
data_vs_valid = DataVS(df_data_valid, df_label_valid)

In [ ]:
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from tqdm import tqdm

class Net(nn.Module):
    def __init__(self, inptut_size, lstm_size, n_classes, num_layers=3):
        super().__init__()

        self.linear = nn.Linear(inptut_size, lstm_size)
        self.lstm = nn.RNN(
            input_size=lstm_size,
            hidden_size=lstm_size,
            num_layers=num_layers,
            dropout=0.2,
            batch_first=True
        )
        self.fc = nn.Linear(lstm_size, n_classes)

    def forward(self, x):
        x = F.relu(self.linear(x))
        x, _ = self.lstm(x)
        x = self.fc(x[:,-1])
        return x

import numpy as np
from sklearn.utils import class_weight

class_weights = class_weight.compute_class_weight('balanced', 
                                                  classes=np.unique(data_vs_train.label[:,-1]), 
                                                  y=data_vs_train.label[:,-1])

samples_weight = np.array([class_weights[t] for t in data_vs_train.label[:,-1]])
samples_weight = torch.from_numpy(samples_weight)

from torch.utils.data.sampler import WeightedRandomSampler
sampler = WeightedRandomSampler(samples_weight.type('torch.DoubleTensor'), len(samples_weight))

trainloader = torch.utils.data.DataLoader(data_vs_train, 128, sampler=sampler)
validloader = torch.utils.data.DataLoader(data_vs_valid, 128, shuffle=False)

net = Net(2, 32, 3)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters())
best_acc = 0

net.to("cuda")
for epoch in range(20):  # loop over the dataset multiple times
    running_acc = 0.0
    total = 0
    correct = 0
    running_loss = 0.0
    net.train()
    
    for i, data in tqdm(enumerate(trainloader, 0), total=len(trainloader)):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        inputs = inputs.to("cuda")
        labels = labels.to("cuda")
        # print(inputs.shape)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        _, predicted = torch.max(outputs.data, 1)
        # print(predicted)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

        # print statistics
        running_loss += loss.item()
        if i == len(trainloader) -1:    # print every 2000 mini-batches
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss/ len(trainloader)}, acc {correct / total}')
            running_loss = 0.0

    running_acc = 0.0
    total = 0
    correct = 0
    running_loss = 0.0
    net.eval()
    for i, data in tqdm(enumerate(validloader, 0), total=len(validloader)):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data
        inputs = inputs.to("cuda")
        labels = labels.to("cuda")
        outputs = net(inputs)

        _, predicted = torch.max(outputs.data, 1)
        # print(predicted)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

        # print statistics
        running_loss += loss.item()
        if i == len(validloader) -1:    # print every 2000 mini-batches
            acc_1 = correct / total
            if best_acc < acc_1:
                print("Save checkpoint")
                torch.save(net, "model.pt")
                best_acc = acc_1
            
            print(f'EVAL loss: {running_loss/ len(validloader)}, acc {acc_1}')
            running_loss = 0.0

total = []
correct = []
from torchmetrics import Accuracy

acc = Accuracy("multiclass", average="macro", num_classes=3)
net = torch.load("model.pt")
net.cpu()
net.eval()
for i, data in tqdm(enumerate(validloader, 0), total=len(validloader)):
    inputs, labels = data
    outputs = net(inputs)
    _, predicted = torch.max(outputs.data, 1)
    total.extend(labels.tolist())
    correct.extend(predicted.tolist())

print("ACC:", acc(torch.tensor(correct),torch.tensor(total)))

print('Finished Training')

In [ ]:

# RNN

# #
# tensor(0.6389) 6531
# #
# tensor(0.6514) 6563
# #
# (0.6067) 6627



# LSTM
# tensor(0.6611) 25539
# ##
#  25571 tensor(0.7096)
# ##
#  25635 tensor(0.6145)